In [1]:
from pyspark.sql.types import ArrayType,FloatType,DataType,BooleanType,StructField,IntegerType,StringType,DateType,StructType

In [2]:
person_schema = StructType([
    StructField("id",IntegerType(),True),
    StructField("first_name",StringType(),True),
    StructField("last_name",StringType(),True),
    StructField("fav_movies",ArrayType(StringType()),True),
    StructField("salary",FloatType(),True),
    StructField("image_url",StringType(),True),
    StructField("date_of_birth",DateType(),True),
    StructField("active",BooleanType(),True)
])

In [3]:
json_path = "C:\zlab\Apache-Spark-3-for-Data-Engineering-and-Analytics-with-Python--main\Section4\persons.json"
df = (spark.read.json(json_path, person_schema, multiLine=True))

In [4]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- fav_movies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- salary: float (nullable = true)
 |-- image_url: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- active: boolean (nullable = true)



In [5]:
df.show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|    Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  2|   Emelyne|    Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...|   1991-11-02| false|
|  3|       Max|   Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|
|  4|    Ilario|     Kean|[Up Close and Per...|3561.36|http://dummyimage...|   1987-06-09|  true|
|  5|     Toddy|   Drexel|[Walk in the Clou...|4934.87|http://dummyimage...|   1992-10-28|  true|
|  6|    Oswald| Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...|   1986-09-02| false|
|  7|    Adrian|   Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...|   1971-08-24| false|
|  8|  Dominica|  Go

## Column and Expression

In [6]:
from pyspark.sql.functions import col,expr

In [7]:
df.select(col("first_name"),col("last_name"),col("date_of_birth")).show(5)

+----------+---------+-------------+
|first_name|last_name|date_of_birth|
+----------+---------+-------------+
|     Drucy|    Poppy|   1991-02-16|
|   Emelyne|    Blaza|   1991-11-02|
|       Max|   Rettie|   1990-03-03|
|    Ilario|     Kean|   1987-06-09|
|     Toddy|   Drexel|   1992-10-28|
+----------+---------+-------------+
only showing top 5 rows



In [8]:
df.select(expr("first_name"),expr("last_name"),expr("date_of_birth")).show(5)

+----------+---------+-------------+
|first_name|last_name|date_of_birth|
+----------+---------+-------------+
|     Drucy|    Poppy|   1991-02-16|
|   Emelyne|    Blaza|   1991-11-02|
|       Max|   Rettie|   1990-03-03|
|    Ilario|     Kean|   1987-06-09|
|     Toddy|   Drexel|   1992-10-28|
+----------+---------+-------------+
only showing top 5 rows



### So sánh lần 1 col và expr khi gọi các cột cụ thể
Ta thấy col và expr không khác gì nhau khi gọi các cột cụ thể<br> Vậy điều gì giúp phân biệt 2 hàm này

In [9]:
from pyspark.sql.functions import concat_ws

In [10]:
(df.select(concat_ws(" ", col("first_name"),col("last_name")).alias("full_name"),
col("salary"),
(col("salary")*0.10+col("salary")).alias("salary_increase"))).show(10)

+----------------+-------+------------------+
|       full_name| salary|   salary_increase|
+----------------+-------+------------------+
|     Drucy Poppy|1463.36|1609.6959838867188|
|   Emelyne Blaza|3006.04|  3306.64404296875|
|      Max Rettie|1422.88|1565.1680053710938|
|     Ilario Kean|3561.36|3917.4961181640624|
|    Toddy Drexel|4934.87|  5428.35712890625|
| Oswald Petrolli|1153.23| 1268.552978515625|
|   Adrian Clarey|1044.73| 1149.202978515625|
|Dominica Goodnow|1147.76|1262.5360107421875|
|   Emory Slocomb|1082.11|1190.3209838867188|
|   Jeremias Bode|3472.63|  3819.89287109375|
+----------------+-------+------------------+
only showing top 10 rows



In [11]:
(df.select(concat_ws(" ", col("first_name"),col("last_name")).alias("full_name"),
col("salary"),
(expr("salary*0.10+salary")).alias("salary_increase"))).show(10)

+----------------+-------+------------------+
|       full_name| salary|   salary_increase|
+----------------+-------+------------------+
|     Drucy Poppy|1463.36|1609.6959838867188|
|   Emelyne Blaza|3006.04|  3306.64404296875|
|      Max Rettie|1422.88|1565.1680053710938|
|     Ilario Kean|3561.36|3917.4961181640624|
|    Toddy Drexel|4934.87|  5428.35712890625|
| Oswald Petrolli|1153.23| 1268.552978515625|
|   Adrian Clarey|1044.73| 1149.202978515625|
|Dominica Goodnow|1147.76|1262.5360107421875|
|   Emory Slocomb|1082.11|1190.3209838867188|
|   Jeremias Bode|3472.63|  3819.89287109375|
+----------------+-------+------------------+
only showing top 10 rows



### sang tới việc ghép chồng first name và last name thành full name và cộng salary
Điểm khác biệt là col tuân theo key-value và muốn lấy giá trị thì bắt buộc phải gọi key thông qua hàm<br>
trong khi đó expr thì không cần, có thể cộng trừ trực tiếp trong chuỗi

# vid 12: 
### filter and where condition

In [12]:
df.filter("salary<3000").show(10)

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|      Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  3|       Max|     Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|
|  6|    Oswald|   Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...|   1986-09-02| false|
|  7|    Adrian|     Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...|   1971-08-24| false|
|  8|  Dominica|    Goodnow|    [Hearts Divided]|1147.76|http://dummyimage...|   1973-08-27| false|
|  9|     Emory|    Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...|   1974-06-08|  true|
| 11|   Timothy|     Ervine|[Land of the Lost...|1147.61|http://dummyimage...|   1971-06-02| false|


In [13]:
df.where("salary<3000").show(10)

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|      Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  3|       Max|     Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|
|  6|    Oswald|   Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...|   1986-09-02| false|
|  7|    Adrian|     Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...|   1971-08-24| false|
|  8|  Dominica|    Goodnow|    [Hearts Divided]|1147.76|http://dummyimage...|   1973-08-27| false|
|  9|     Emory|    Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...|   1974-06-08|  true|
| 11|   Timothy|     Ervine|[Land of the Lost...|1147.61|http://dummyimage...|   1971-06-02| false|


ta thấy rằng với những lệnh cơ bản thì nó sẽ cho những kết quả khá giống nhau<br>
giờ ta sẽ tới với các lệnh nâng cao để so sánh sự khác biệt

In [14]:
df.where((col("salary")<=3000) & (col("active")=="true")).show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|    Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  9|     Emory|  Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...|   1974-06-08|  true|
| 16|   Margaux| Archbold|[And Now a Word f...|1013.75|http://dummyimage...|   1988-07-29|  true|
| 26|     Clive|      Lax|             [Rabid]|2126.87|http://dummyimage...|   1981-10-26|  true|
| 33|  Sherline|  Primett|   [Jungle Fighters]|2309.39|http://dummyimage...|   1972-07-23|  true|
| 34|     Davis|    Pinks|          [Hounddog]|1337.14|http://dummyimage...|   1989-07-27|  true|
| 37|    Carlen|  Sharply|[Dr. Jekyll and M...|2051.85|http://dummyimage...|   2002-06-01|  true|
| 40|    Jordan|   L

In [15]:
from pyspark.sql.functions import year

In [16]:
df.filter((year("date_of_birth")==2000) | (year("date_of_birth")==1989)).show(10)

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| 14|   Ambrosi|   Vidineev|[Wall Street: Mon...|4550.88|http://dummyimage...|   1989-07-20|  true|
| 15|    Feodor|Nancekivell|   [Monsoon Wedding]|2218.46|http://dummyimage...|   2000-10-07| false|
| 18|     Alfie|   Hatliffe|     [Lord of Tears]| 3893.1|http://dummyimage...|   1989-06-21|  true|
| 25|     Kelcy|     Wogdon|    [Iron Mask, The]|4512.51|http://dummyimage...|   2000-10-20|  true|
| 32|      Redd|   Akenhead|[Century of the D...| 2470.9|http://dummyimage...|   2000-06-05| false|
| 34|     Davis|      Pinks|          [Hounddog]|1337.14|http://dummyimage...|   1989-07-27|  true|
| 61|    Shanna|    Samples|[Thomas in Love (...| 2703.0|http://dummyimage...|   1989-07-07| false|


In [17]:
from pyspark.sql.functions import array_contains

In [19]:
df.where(array_contains(df.fav_movies,"a")).show()

+---+----------+---------+----------+------+---------+-------------+------+
| id|first_name|last_name|fav_movies|salary|image_url|date_of_birth|active|
+---+----------+---------+----------+------+---------+-------------+------+
+---+----------+---------+----------+------+---------+-------------+------+



# Distinct, Drop Duplicates, Order By

In [23]:
from pyspark.sql.functions import count, desc

In [24]:
df.select("active").distinct().show()

+------+
|active|
+------+
|  true|
| false|
+------+



In [26]:
df.select(col("first_name"),
         year(col("date_of_birth")).alias("year"),
         col("active")).orderBy("year","first_name").show(10)

+----------+----+------+
|first_name|year|active|
+----------+----+------+
|    Adrian|1971| false|
|   Feodora|1971|  true|
|       Sky|1971| false|
|   Timothy|1971| false|
|    Lucita|1972|  true|
|      Rodi|1972| false|
|  Sherline|1972|  true|
|     Toddy|1972|  true|
|  Dominica|1973| false|
|    Kelila|1973|  true|
+----------+----+------+
only showing top 10 rows



In [28]:
df.select(col("first_name"),
         year(col("date_of_birth")).alias("year"),
         col("active")).dropDuplicates(["year","active"]).orderBy("year","first_name").show(10)

+----------+----+------+
|first_name|year|active|
+----------+----+------+
|    Adrian|1971| false|
|   Feodora|1971|  true|
|      Rodi|1972| false|
|  Sherline|1972|  true|
|  Dominica|1973| false|
|    Kelila|1973|  true|
|   Balduin|1974| false|
|     Emory|1974|  true|
|    Janean|1975|  true|
|       Bev|1976|  true|
+----------+----+------+
only showing top 10 rows



# Row and Union

In [30]:
from pyspark.sql import Row

Thêm một hàng

In [31]:
person_row = Row(101,"Robert","Ownes",["Men in black III","Home Alone"],4340.5,"http://imagesss.com","1964-08-18",True)

In [32]:
row_list = [Row(102,"do","kien",["Men in black III","Home Alone"],4340.5,"http://imagesss.com","1964-08-18",True),
           Row(103,"nguyen","thi",["Men in black III","Home Alone"],4340.5,"http://imagesss.com","1964-08-18",True),
           Row(104,"van","anh",["Men in black III","Home Alone"],4340.5,"http://imagesss.com","1964-08-18",True)]

In [34]:
row_list.append(person_row)

In [35]:
print(row_list)

[<Row(102, 'do', 'kien', ['Men in black III', 'Home Alone'], 4340.5, 'http://imagesss.com', '1964-08-18', True)>, <Row(103, 'nguyen', 'thi', ['Men in black III', 'Home Alone'], 4340.5, 'http://imagesss.com', '1964-08-18', True)>, <Row(104, 'van', 'anh', ['Men in black III', 'Home Alone'], 4340.5, 'http://imagesss.com', '1964-08-18', True)>, <Row(101, 'Robert', 'Ownes', ['Men in black III', 'Home Alone'], 4340.5, 'http://imagesss.com', '1964-08-18', True)>]
